In [1]:
import os
import numpy as np 
from glob import glob
from PIL import Image
import cv2
import slideio
import json
from skimage.draw import polygon2mask
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
CLS_whi_list=glob('../../data/1-cycle_30%_중간데이터/1_insitu/WSI/*.tiff')
normal_whi_list=glob('../../data/1-cycle_30%_중간데이터/0_normal/WSI/*.tiff')
abnormal_whi_list=glob('../../data/1-cycle_30%_중간데이터/2_malignant/WSI/*.tiff')

image_path='../../data/1-cycle_30%_중간데이터/segmentation/image/'
CLS_mask_path='../../data/1-cycle_30%_중간데이터/classification/polygon/1_insitu/'
normal_mask_path='../../data/1-cycle_30%_중간데이터/classification/polygon/0_normal/'
abnormal_mask_path='../../data/1-cycle_30%_중간데이터/classification/polygon/2_malignant/'


In [3]:
def size_ratio(scene,img_size):
    width = scene.rect[2]
    height = scene.rect[3]
    ratio=0
    inverse_ratio=0
    img_width=0
    img_height=0
    if width>height:
        ratio=img_size/width
        inverse_ratio=width/img_size
        img_width=img_size
        img_height=height*ratio
    else:
        ratio=img_size/height
        inverse_ratio=height/img_size
        img_height=img_size
        img_width=width*ratio
        
    return int(img_width),int(img_height),inverse_ratio

seg

In [4]:
json_list=[f.replace('/WSI/', '/json/') for f in CLS_whi_list]
json_list=[f.replace('.tiff', '.json') for f in json_list]
for i in tqdm(range(len(CLS_whi_list))):
    slide = slideio.open_slide(CLS_whi_list[i], "GDAL")
    fileName=os.path.basename(os.path.splitext(CLS_whi_list[i])[0])
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    img_width,img_height,ratio=size_ratio(scene,2048)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(img_width), int(img_height)))
    
    dst_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    with open(json_list[i]) as f:
        json_object = json.load(f)
    polygon_count=len(json_object['files'][0]['objects'])
    image_shape=(img_height,img_width)
    for j in range(polygon_count):
        if json_object['files'][0]['objects'][j]['label']=='TP_tumor':
            polygon=np.array(json_object['files'][0]['objects'][j]['coordinate'])*1/ratio
            polygon1=np.copy(polygon)
            polygon1[:,0]=polygon[:,1]
            polygon1[:,1]=polygon[:,0]
            mask=polygon2mask(image_shape,polygon1)
            dst_mask=mask+dst_mask
    dst_mask=np.where(dst_mask>0,255,0)
    dst_mask=cv2.cvtColor(dst_mask.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    cv2.imwrite(CLS_mask_path+fileName+'.tiff', dst_mask )

  0%|          | 0/1485 [00:00<?, ?it/s]

In [5]:
json_list=[f.replace('/WSI/', '/json/') for f in normal_whi_list]
json_list=[f.replace('.tiff', '.json') for f in json_list]
for i in tqdm(range(len(normal_whi_list))):
    slide = slideio.open_slide(normal_whi_list[i], "GDAL")
    fileName=os.path.basename(os.path.splitext(normal_whi_list[i])[0])
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    img_width,img_height,ratio=size_ratio(scene,2048)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(img_width), int(img_height)))
    
    dst_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    with open(json_list[i]) as f:
        json_object = json.load(f)
    polygon_count=len(json_object['files'][0]['objects'])
    image_shape=(img_height,img_width)
    for j in range(polygon_count):
        if json_object['files'][0]['objects'][j]['label']=='NT_normal':
            polygon=np.array(json_object['files'][0]['objects'][j]['coordinate'])*1/ratio
            polygon1=np.copy(polygon)
            polygon1[:,0]=polygon[:,1]
            polygon1[:,1]=polygon[:,0]
            mask=polygon2mask(image_shape,polygon1)
            dst_mask=mask+dst_mask
    dst_mask=np.where(dst_mask>0,255,0)
    dst_mask=cv2.cvtColor(dst_mask.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    cv2.imwrite(normal_mask_path+fileName+'.tiff', dst_mask )

  0%|          | 0/1485 [00:00<?, ?it/s]

In [6]:
json_list=[f.replace('/WSI/', '/json/') for f in abnormal_whi_list]
json_list=[f.replace('.tiff', '.json') for f in json_list]
for i in tqdm(range(len(abnormal_whi_list))):
    slide = slideio.open_slide(abnormal_whi_list[i], "GDAL")
    fileName=os.path.basename(os.path.splitext(abnormal_whi_list[i])[0])
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    img_width,img_height,ratio=size_ratio(scene,2048)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(img_width), int(img_height)))
    dst_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    with open(json_list[i]) as f:
        json_object = json.load(f)
    polygon_count=len(json_object['files'][0]['objects'])
    image_shape=(img_height,img_width)
    for j in range(polygon_count):
        if json_object['files'][0]['objects'][j]['label']=='TP_tumor':
            polygon=np.array(json_object['files'][0]['objects'][j]['coordinate'])*1/ratio
            polygon1=np.copy(polygon)
            polygon1[:,0]=polygon[:,1]
            polygon1[:,1]=polygon[:,0]
            mask=polygon2mask(image_shape,polygon1)
            dst_mask=mask+dst_mask
    dst_mask=np.where(dst_mask>0,255,0)
    dst_mask=cv2.cvtColor(dst_mask.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    cv2.imwrite(abnormal_mask_path+fileName+'.tiff', dst_mask )

  0%|          | 0/1765 [00:00<?, ?it/s]

cla

In [7]:
negative_whi_list=glob('../../data/1-cycle_10%_초기데이터/방광암/tiff/정상(비종양)/*.tiff')
positive_whi_list=glob('../../data/1-cycle_10%_초기데이터/방광암/tiff/종양(악성)/*.tiff')
CIS_whi_list=glob('../../data/1-cycle_10%_초기데이터/방광암/tiff/제자리암종/*.tiff')
f_path='../../data/1-cycle_10%_초기데이터/classification/negative/'
for i in range(len(negative_whi_list)):
    slide = slideio.open_slide(negative_whi_list[i], "GDAL")
    num_scenes = slide.num_scenes
    
    scene = slide.get_scene(0)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(512), int(512)))
    image=cv2.cvtColor(slide_block, cv2.COLOR_BGR2RGB)
    dst_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    cv2.imwrite(f_path+str(i)+'.tiff', image)
    
f_path='../../data/1-cycle_10%_초기데이터/classification/positive/'
for i in range(len(positive_whi_list)):
    slide = slideio.open_slide(positive_whi_list[i], "GDAL")
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(512), int(512)))
    image=cv2.cvtColor(slide_block, cv2.COLOR_BGR2RGB)
    dst_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    cv2.imwrite(f_path+str(i)+'.tiff', image)
    
f_path='../../data/1-cycle_10%_초기데이터/classification/CIS/'
for i in range(len(CIS_whi_list)):
    slide = slideio.open_slide(CIS_whi_list[i], "GDAL")
    num_scenes = slide.num_scenes
    scene = slide.get_scene(0)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    slide_block = scene.read_block((0, 0, svsWidth, svsHeight),size=(int(512), int(512)))
    image=cv2.cvtColor(slide_block, cv2.COLOR_BGR2RGB)
    dst_mask=np.zeros((img_height,img_width),dtype=np.uint8)
    cv2.imwrite(f_path+str(i)+'.tiff', image)